In [1]:
from urllib2 import urlopen
from bs4 import BeautifulSoup
import re, time
import numpy as np
import pandas as pd
import requests
import sys
import json

#cdd4868100681033 API key

In [2]:
def getValue(sourceTitle,soup):
    segmentSoup = soup.find("div", sourceTitle)
    valSoup     = BeautifulSoup(str(segmentSoup), "lxml")
    val         = valSoup.find("span", "value")
    if val == None:
        return "missing"
    return val.string

In [31]:
def findAnnualPrecipitation(year, city):
    urlA = 'http://www.wunderground.com/history/airport/KSEA/'
    urlB = '/12/31/DailyHistory.html?req_city='
    urlC = '&req_state=WA&req_statename=Washington&reqdb.zip=98446&reqdb.magic=1&reqdb.wmo=99999'
    url  = urlA+str(year)+urlB+city+urlC
    print url
#     try:
#         url  = urlA+str(year)+urlB+city+urlC
#     except ConnectionError:
#         return "missing"
    page = requests.get(url).text
    pIndex = page.index("Year to date precipitation") #index in urlpage where the string is found
    prepContent = page[pIndex+50:pIndex+72] #gets the entire value required in html format
    valSoup     = BeautifulSoup(str(prepContent), "lxml")
    val         = valSoup.find("td", "")
    if val == None:
        return "missing"
    return val.string

In [121]:
url = 'http://www.wunderground.com/history/airport/KSEA/1999/2/1/MonthlyHistory.html?req_city=Auburn&req_state=WA&req_statename=Washington&reqdb.zip=98001&reqdb.magic=1&reqdb.wmo=99999&MR=1'
page = requests.get(url).text

In [144]:
def getMonthWeather(city, zipcode, year, month):
    wBank = {}
    part1 = 'http://www.wunderground.com/history/airport/KSEA/'
    part2 = str(year)+'/'+str(month)+'/1/MonthlyHistory.html?req_city='
    part3 = city + '&req_state=WA&req_statename=Washington&reqdb.zip='
    part4 = str(zipcode) + '&reqdb.magic=1&reqdb.wmo=99999'
    url = part1 + part2 + part3 + part4
    page = requests.get(url).text
    snowdays, events = getEvents(page)
    maxTemps = getData("Max Temperature", 400)
    meanTemps = getData("Mean Temperature", 400)
    minTemps = getData("Min Temperature", 400)
    preci = getData("Precipitation</span>", 450)
    key   = city+'-'+str(year)+'-'+str(month)
    wBank[key] = {'MaxTemp' : maxTemps, 'MeanTemp' : meanTemps, 'MinTemp' : minTemps, 
                  'Precipitation' : preci, 'Snowed':snowDays, 'DailyEvents':events}
    return wBank

In [103]:
# text = [str(i.string) for i in table.findAll(text=lambda(x): len(x) >4)]

In [152]:
test1 = getMonthWeather('Cheney', 99004, 1990, 8)

In [153]:
print test1

{'Cheney-1990-8': {'MeanTemp': ['49', '42', '34'], 'MaxTemp': ['53', '47', '37'], 'MinTemp': ['45', '38', '30'], 'DailyEvents': ['Fog', 'Fog', 'Rain', 'Rain', 'Rain', 'Fog', 'Rain', 'Fog', 'Rain', 'Fog', 'Rain', 'Rain', 'Fog', 'Rain'], 'Snowed': 3, 'Precipitation': ['1.04', '0.24', '0.00', '6.84']}}


In [120]:
def getEvents(page):
    soup = BeautifulSoup(page, "lxml")
    table = soup.findAll("table")[3]
    text = re.findall("\\n[a-zA-Z]+\\n[\\t]?", str(table))
    events = [re.findall("[a-zA-Z]+", event)[0] for event in text]
    snowDays = events.count("Snow")
    return snowDays, events

In [123]:
snowDays, Events = getEvents(page)

In [33]:
def getData(name, nx):
    try:
        startInd = page.index(name)
    except ValueError:
        return "Not found"
    content  = page[startInd+30 : startInd+nx]
    valSoup  = BeautifulSoup(str(content), "lxml")
    values   = valSoup.findAll("span", "wx-value")
    return [str(val.string) for val in values]

In [36]:
minTemps = getData("Min Temperature", 400)

In [31]:
preci = getData("Precipitation</span>", 450)

In [32]:
print preci

['1.40', '0.16', '0.00', '5.01']


In [4]:
Aub1980 = findAnnualPrecipitation(1980, 'Auburn')

In [5]:
print Aub1980

37.49


In [6]:
def getAnnualPrec(city):
    values = []
    for year in np.arange(20)+1980:
        aPrec = findAnnualPrecipitation(year, city)
        values.append(aPrec)
        time.sleep(10)
    return values

In [7]:
# AubPrec = getAnnualPrec('Auburn')

In [8]:
# print AubPrec

In [9]:
xl_file = pd.ExcelFile('zip_code_database.xls')

In [10]:
dfWSzip = xl_file.parse('Sheet1') 

In [11]:
dfWSzip.head()

,type,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,latitude,longitude,world_region,country,decommissioned,estimated_population,notes
98001,STANDARD,Auburn,"Algona, Federal Way",NaN,WA,King County,America/Los_Angeles,206253,47.30,-122.26,NaN,US,0,26341,NaN
98002,STANDARD,Auburn,NaN,NaN,WA,King County,America/Los_Angeles,253206,47.31,-122.20,NaN,US,0,23795,NaN
98003,STANDARD,Federal Way,Auburn,NaN,WA,King County,America/Los_Angeles,206253,47.30,-122.31,NaN,US,0,33131,NaN
98004,STANDARD,Bellevue,"Beaux Arts, Clyde Hill, Hunts Point, Yarrow Point",Beaux Arts Village,WA,King County,America/Los_Angeles,425206253,47.61,-122.21,NaN,US,0,23399,NaN
98005,STANDARD,Bellevue,NaN,NaN,WA,King County,America/Los_Angeles,425206,47.62,-122.16,NaN,US,0,15676,NaN


In [12]:
cities = np.unique(np.array(dfWSzip.loc[:,'primary_city']))

In [13]:
cities = [str(city) for city in cities]

In [14]:
def prep(cityList):
    dic = {}
    for city in cityList:
        annualPreps = getAnnualPrec(city)
        dic[city] = annualPreps
    return dic

In [15]:
print cities[:10]

['Aberdeen', 'Acme', 'Addy', 'Adna', 'Airway Heights', 'Albion', 'Allyn', 'Almira', 'Amanda Park', 'Amboy']


In [37]:
print findAnnualPrecipitation(1999, 'Tacoma')

http://www.wunderground.com/history/airport/KSEA/1999/12/31/DailyHistory.html?req_city=Tacoma&req_state=WA&req_statename=Washington&reqdb.zip=98446&reqdb.magic=1&reqdb.wmo=99999
37.49


In [177]:
f = urllib2.urlopen('http://api.wunderground.com/api/cdd4868100681033/history_20100208/q/WA/Cheney.json')
json_string = f.read()
# parsed_json = json.loads(json_string)
# location = parsed_json['location']['city']
# temp_f = parsed_json['current_observation']['temp_f']
# print "Current temperature in %s is: %s" % (location, temp_f)
f.close()

In [2]:
url = '7&yearend=2014&format=1'

In [3]:
page = requests.get(url).text

In [40]:
# print page

In [4]:
data = page.split("<br />")

In [5]:
colnames = [str(name) for name in data[0].split(",")]

In [25]:
dic = {}
for i in data[1:-1]:
    item = i.split(",")
    dic[str(item[0])[1:]] = [str(a) for a in item[1:]]

In [28]:
print dic['2014-7-31']

['86', '68', '50', '55', '50', '42', '94', '52', '21', '30.13', '30.09', '30.02', '10', '10', '10', '14', '5', '18', '0.00', '0', '', '35']


In [30]:
df = pd.DataFrame.from_dict(dic, orient = "index")


In [32]:
print len(df)

365


In [38]:
df.columns = colnames[1:]

In [49]:
def weatherHist(airportIndex, year):
    a = 'http://api.wunderground.com/history/airport/'
    b = airportIndex +'/'+ str(year)+'/1/1/CustomHistory.html?dayend=31&monthend=12&yearend='
    c = str(year)+'&format=1'
    url = a+b+c
    page = requests.get(url).text
    data = page.split("<br />")
    dic = {}
    for i in data[1:-1]:
        item = i.split(",")
        dic[str(item[0])[1:]] = [str(a) for a in item[1:]]
    df = pd.DataFrame.from_dict(dic, orient = "index")
    colnames = [str(name) for name in data[0].split(",")]
    df.columns = colnames[1:]
    return df

In [50]:
df1 = weatherHist('KSEA', 1981)

In [57]:
print df1.head()

           Max TemperatureF Mean TemperatureF Min TemperatureF Max Dew PointF  \
1981-12-9                50                46               41             46   
1981-12-8                42                40               37             38   
1981-11-30               46                41               37             40   
1981-12-7                48                45               39             41   
1981-12-6                46                44               43             43   

           MeanDew PointF Min DewpointF Max Humidity  Mean Humidity  \
1981-12-9              41            37           90             84   
1981-12-8              36            34           93             86   
1981-11-30             36            33           85             79   
1981-12-7              38            36           89             76   
1981-12-6              41            39           90             85   

            Min Humidity  Max Sea Level PressureIn       ...        \
1981-12-9       

In [51]:
code = "test"

In [71]:
def saveDF(code, df):
    name = 'WeatherHistWA/'+code+'.xlsx'
    writer = pd.ExcelWriter(name, engine='xlsxwriter')
    df.to_excel(writer,'Sheet1') # change the dateframe respectively
    writer.save()

In [63]:
import xlsxwriter #import this first!!

In [72]:
saveDF(code, df1)

In [73]:
type(df1)

pandas.core.frame.DataFrame

In [74]:
type(code)

str

In [78]:
def checkType(item):
    if type(item) != str:
        print "okay"
    print "NIce"

In [80]:
checkType(df1)

okay
NIce
